# Laboratorio Módulo 3: Deployment y Consumo de LLaMa 3.1 con Ollama

## Llama 3.1 - Capacitación SICYT Argentina 2025

**Capacitación:** SICYT Argentina 2025  
**Fecha:** Octubre 2025

**Objetivo:**  
Aprender a instalar y usar Ollama para ejecutar modelos LLaMa en CPU y consumir el modelo desde Python en Visual Studio Code usando la API REST. Incluye instalación de herramientas y ejercicios prácticos.

---

## 0. Instalación de Herramientas

### 0.1 Instalar Visual Studio Code

- Descargar Visual Studio Code desde: [https://code.visualstudio.com/](https://code.visualstudio.com/)
- Instalar como cualquier programa en Windows o Mac.

### 0.2 Instalar Python 3.10 o superior

- Descargar Python desde: [https://www.python.org/downloads/](https://www.python.org/downloads/)
- Durante la instalación en Windows, marcar la opción **"Add Python to PATH"**.
- En Mac, puedes usar el instalador o `brew install python`.

### 0.3 Instalar extensiones en VS Code

1. Abre VS Code.
2. Ve a la barra lateral izquierda (ícono de cuadrados) o presiona `Ctrl+Shift+X`.
3. Busca e instala:
    - **Python** (de Microsoft)
    - **Jupyter** (de Microsoft)

---

## 1. Preparación del entorno de trabajo

### 1.1 Crear y activar un entorno virtual (opcional pero recomendado)

Abre la terminal integrada en VS Code (`Ctrl+ñ` o desde menú Terminal > Nueva terminal).

#### **Windows:**
```bash
python -m venv venv
.\venv\Scripts\activate
```

#### **Mac:**
```bash
python3 -m venv venv
source venv/bin/activate
```

### 1.2 Instalar dependencias Python
```bash
pip install jupyter requests
```

---

## 2. Instalación y prueba de Ollama

### 2.1 Instalar Ollama

- **Windows:**  
  - Descarga el instalador desde [https://ollama.ai/download](https://ollama.ai/download) y sigue las instrucciones. Puede requerir WSL2 (se instala automáticamente si es necesario).
- **Mac:**  
  - Puedes instalar desde terminal con:
    ```bash
    brew install ollama
    ```
    O descarga el instalador desde [https://ollama.ai/download](https://ollama.ai/download).

### 2.2 Verificar la instalación
Abre una terminal (puede ser la misma de VS Code) y ejecuta:
```bash
ollama --version
```

---

## 3. Descargar y ejecutar LLaMa 3.1 en Ollama

### 3.1 Descargar modelo
```bash
ollama pull llama3
```

### 3.2 Probar el modelo desde la terminal
```bash
ollama run llama3 "¿Qué es la inteligencia artificial?"
```

---

## 4. Consumir Ollama desde Python usando la API REST
> Antes de seguir, asegúrate que Ollama está corriendo (el instalador inicia el servicio automáticamente; si no, ejecuta `ollama serve`).

In [ ]:
import requests

OLLAMA_HOST = "http://127.0.0.1:11434"

prompt = "Explica brevemente el concepto de aprendizaje profundo."
payload = {
    "model": "llama3.1",
    "prompt": prompt,
    "stream": False
}

response = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload)
result = response.json()
print("Respuesta del modelo:")
print(result.get("response", "Error"))

---

### 4.2 Experimenta con diferentes parámetros

In [ ]:
for temperature in [0.3, 0.7, 1.0]:
    payload = {
        "model": "llama3.1",
        "prompt": "Resume la importancia de los transformers en el procesamiento de lenguaje natural.",
        "temperature": temperature,
        "top_p": 0.9,
        "stream": False
    }
    response = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload)
    print(f"\nTemperature={temperature}:")
    print(response.json().get("response", "Error"))

---

### 4.3 Probar streaming de resultados

In [ ]:
import json

payload = {
    "model": "llama3",
    "prompt": "Nombra tres aplicaciones prácticas de los LLMs en empresas.",
    "stream": True
}
response = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, stream=True)
print("Respuesta en streaming:")
for line in response.iter_lines():
    if line:
        chunk = json.loads(line)
        print(chunk.get("response", ""), end="", flush=True)
        if chunk.get("done"):
            print("\n--- Generación completada ---")

---

## 5. Manejo básico de errores en la API

In [ ]:
try:
    payload = {
        "model": "llama3",
        "prompt": "¿Cuál es el futuro de la IA?",
        "stream": False
    }
    response = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=40)
    response.raise_for_status()
    print(response.json().get("response", ""))
except requests.exceptions.ConnectionError:
    print("Error: Ollama no está corriendo en localhost:11434")
except Exception as e:
    print("Error inesperado:", e)

Error inesperado: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=40)


---

## 6. (Opcional) Mini servidor Flask para consumir Ollama desde web

**Ejecuta este script por separado si tienes Flask instalado**

In [ ]:
# Guardar como app.py y ejecutar con: python app.py
from flask import Flask, request, jsonify
import requests

OLLAMA_HOST = "http://localhost:11434"

app = Flask(__name__)

@app.route('/genera', methods=['POST'])
def genera():
    datos = request.json
    prompt = datos.get("prompt", "")
    payload = {
        "model": "llama3.1",
        "prompt": prompt,
        "stream": False
    }
    resp = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload)
    return jsonify(resp.json())

if __name__ == "__main__":
    app.run(port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

---

## 7. Preguntas y ejercicios de reflexión

1. **¿Qué diferencias observas en las respuestas del modelo cuando cambias el parámetro `temperature`?**
2. **¿Cómo podrías integrar Ollama en una aplicación real de atención al cliente?**
3. **¿Qué limitaciones encontraste usando la API REST de Ollama?**
4. **Prueba agregar un endpoint extra en Flask que realice streaming de la respuesta (requiere manejo avanzado de streams en Flask).**

---

**¡Fin del laboratorio!**
